## Importing Libraries

In [ ]:
! pip install gensim --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os
sys.path.append('/content/drive/MyDrive/quora_duplicate_questions/src')

import joblib
import re

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import (cosine_similarity,
                                      euclidean_distances,
                                      manhattan_distances)
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import gensim.downloader as api

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

from tqdm.auto import tqdm
tqdm.pandas()

from utils import evaluate_model, plot_confusion_matrix

## Load data

In [ ]:
DATA_PATH = '/content/drive/MyDrive/quora_duplicate_questions/data/processed'
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv.zip'), compression='zip')
val_df = pd.read_csv(os.path.join(DATA_PATH, 'val.csv.zip'), compression='zip')

## Preprocess text

In [ ]:
def preprocess_text(text):
    """Clean and preprocess text"""
    if pd.isna(text):
        return []

    # Convert to lowercase
    text = str(text).lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    stop_words = set(stopwords.words('english'))

    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

In [ ]:
print("Preprocessing questions for Train set...")
train_df['q1_tokens'] = train_df['question1'].progress_apply(preprocess_text)
train_df['q2_tokens'] = train_df['question2'].progress_apply(preprocess_text)

In [ ]:
print("Preprocessing questions for Validation set...")
val_df['q1_tokens'] = val_df['question1'].progress_apply(preprocess_text)
val_df['q2_tokens'] = val_df['question2'].progress_apply(preprocess_text)

## Load GloVe Embeddings

In [ ]:
glove_name='glove-wiki-gigaword-100'

glove_model = api.load(glove_name)
print(f"GloVe model loaded successfully. Vocabulary size: {len(glove_model)}")
print(f"Embedding dimension: {glove_model.vector_size}")

In [ ]:
def get_sentence_embedding(tokens, model):
    """Get sentence embedding by averaging word embeddings"""
    if not tokens:
      return np.zeros(model.vector_size)

    embeddings = []
    for token in tokens:
        if token in model:
            embeddings.append(model[token])

    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
      return np.zeros(model.vector_size)

## Prepare text features

In [ ]:
def create_features(df, model):
    """Create features using embeddings"""
    features = []

    for idx, row in df.iterrows():
        if idx % 10000 == 0:
            print(f"Precessing row {idx}/{len(df)}")

        # Get embeddings for each question
        q1_emb = get_sentence_embedding(row['q1_tokens'], model)
        q2_emb = get_sentence_embedding(row['q2_tokens'], model)

        # Create similarity features
        cos_sim = cosine_similarity(q1_emb.reshape(1, -1), q2_emb.reshape(1, -1))[0][0]
        euclidean_dist = euclidean_distances(q1_emb.reshape(1, -1),
                                             q2_emb.reshape(1, -1))[0][0]
        manhattan_dist = manhattan_distances(q1_emb.reshape(1, -1),
                                             q2_emb.reshape(1, -1))[0][0]

        # Element-wise features
        diff = q1_emb - q2_emb
        mult = q1_emb * q2_emb

        # Combine features
        feature_vector = np.concatenate([
            q1_emb, q2_emb, diff, mult, [cos_sim], [euclidean_dist], [manhattan_dist]
        ])
        features.append(feature_vector)

    return np.array(features)

In [ ]:
print("Create features for Train set:")
X_train_features = create_features(train_df, glove_model)

print("\nCreate features for Validation set:")
X_val_features = create_features(val_df, glove_model)

## Train and Evaluate Logistic Regression

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
X_val_scaled = scaler.transform(X_val_features)

In [ ]:
y_train = train_df['is_duplicate']
y_val = val_df['is_duplicate']

In [ ]:
classifier = LogisticRegression(random_state=7, class_weight='balanced', max_iter=1000)
classifier.fit(X_train_scaled, y_train)

In [ ]:
log_loss_train, log_loss_val = evaluate_model(
    X_train_scaled, X_val_scaled, y_train, y_val, classifier
)
print(f"Log loss on Train set: {log_loss_train:.4f}")
print(f"Log loss on Validation set: {log_loss_val:.4f}")

In [ ]:
print(f"Classification Report:")
y_pred = classifier.predict(X_val_scaled)
print(classification_report(y_val, y_pred,
                            target_names=['Not Duplicate', 'Duplicate']))


plot_confusion_matrix(y_val, y_pred)

In [ ]:
experiment_results = joblib.load(os.path.join(DATA_PATH, 'experiment_results.joblib'))

In [ ]:
results = {}
results['model'] = 'GloVe Embeddings with Logistic Regression'
results['log_loss_train'] = np.round(log_loss_train, 5)
results['log_loss_val'] = np.round(log_loss_val, 5)

experiment_results.append(results)

In [ ]:
joblib.dump(experiment_results, os.path.join(DATA_PATH, 'experiment_results.joblib'))

`experiment_results` is updated and saved with the latest experiment's metrics.